In [2]:
!pip install accelerate>=0.26.0
import pandas as pd 
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
import torch
import transformers

In [3]:
# Assuming your filtered dataset CSV (form pross_date.py) is saved here:
data_path = "../data/quantum_subset.csv"
df = pd.read_csv(data_path)

#Keep only neccessary columns 
df = df[['title', 'categories']]

# encode catagories
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['lable'] = le.fit_transform(df['categories'])

# Split 
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['title'].tolist(), df['lable'].tolist(), test_size=0.2, random_state=42
)


In [4]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=64)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=64)



In [5]:
class ArxivDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item
    
    def __len__(self):
        return len(self.labels)
    
train_dataset = ArxivDataset(train_encodings, train_labels)
val_dataset = ArxivDataset(val_encodings, val_labels)


In [7]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=len(le.classes_))
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="../models/category_classifier/",
    
    evaluation_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=2,
    save_strategy="epoch",
    logging_dir="../evaluation/",

)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)
trainer.train()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


TypeError: TrainingArguments.__init__() got an unexpected keyword argument 'evaluation_strategy'